# Toy problem: retrieve infeasibility info from gurobi when solving QPs

Consider the infeasible QP
\begin{align}
\min \frac{1}{2} x_1^2
\quad
\text{subject to}
\quad
x_0 \geq 0,
x_1 \geq 0,
x_0 + x_1 \leq -1,
\end{align}
and its unbounded dual
\begin{align}
\max - \frac{1}{2} (p_1-q)^2 + q
\quad
\text{subject to}
\quad
p_0 \geq 0,
p_1 \geq 0,
q \geq 0,
p_0 = q.
\end{align}

How to retrieve from Gurobi the multipliers $(p_0, p_1, q) = (\alpha, \alpha, \alpha)$ with $\alpha > 0$ which prove unboundedness of the dual, and hence infeasibility of the primal?

It seems that this info can be retrieved only in case of linear programs.
However, the same multipliers that prove infeasibility of the LP also prove infeasibility of the QP.
Hence it is sufficient to set to objective to zero and run the LP.

Note that the dual of the LP
\begin{align}
\min 0
\quad
\text{subject to}
\quad
x_0 \geq 0,
x_1 \geq 0,
x_0 + x_1 \leq -1,
\end{align}
is
\begin{align}
\max q
\quad
\text{subject to}
\quad
p_0 \geq 0,
p_1 \geq 0,
q \geq 0,
p_0 = q,
p_1 = q.
\end{align}

In [127]:
import gurobipy as grb

In [128]:
m = grb.Model()
x = m.addVars(2, lb=[-grb.GRB.INFINITY]*2)
m.setObjective(x[0] + x[1])

# bounds are expressed as constraints otherwise we cannot get their multipliers out of the Farkas proof
lb0 = m.addConstr(- x[0] <= 0., name='') # express the constraints with <= so that multipliers are positive
lb1 = m.addConstr(- x[1] <= 0.)
c = m.addConstr(x[0] + x[1] <= -1.)
c_redundant = m.addConstr(- x[0] - x[1] <= 1.) # redundant

In [129]:
m.Params.InfUnbdInfo = 1 # to query the infeasibility proof (only LP)
m.optimize()

Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 2 iterations and 0.02 seconds
Infeasible model


In [130]:
# retrieve duals which prove infeasibility for the LP
print m.FarkasDual

[1.0, 1.0, 1.0, 0.0]


In [117]:
# note that calling the duals directly gives garbage
print [lb0.Pi, lb1.Pi, c.Pi, c_redundant.Pi]

[-1.0, -1.0, 0.0, 0.0]


In [121]:
c = m.getConstrs()

In [123]:
aa = c[0]

In [126]:
aa.FarkasDual

1.0

In [ ]:
constraints['gatto'] = 1

In [ ]:
constraints

In [ ]:
aaa = ('u', 22, 3)

In [ ]:
lb0.RHS = 22.
m.update()

In [ ]:
lb0.

In [ ]:
x = m.addVars(10, name='x')
m.update()

In [ ]:
X = np.vstack((
    np.hstack((np.eye(6), np.zeros((6,3)))),
    np.hstack((np.zeros((2,6)), np.zeros((2,3))))
))
Y = np.vstack((
    np.hstack((np.eye(4), np.zeros((4,1)))),
    np.hstack((np.zeros((5,4)), np.zeros((5,1))))
))

In [ ]:
print X.shape, np.linalg.matrix_rank(X)
print Y.shape, np.linalg.matrix_rank(Y)

In [ ]:
np.linalg.matrix_rank(X.dot(Y))

In [ ]:
G = np.array([[0., 0., 1., 0.],
              [1., 1., 0., 0.],
              [1., 0., 0., 0.],
              [1., 1., 0., 0.]])
print np.linalg.matrix_power(G,9)

In [ ]:
G = np.array([[0, 0, 1],
              [1, 0, 0],
              [0, 1, 0]])
np.linalg.eig(G)[0]

In [ ]:
for i in range(40):
    A = np.random.choice([0, 1], size=(3,3))
    eigs = np.linalg.eig(A)[0]
    for eig in eigs:
        if isinstance(eig, complex):
            print False, A, eig

In [89]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
from pympc.control.hybrid_benchmark.utils import GurobiModel
import numpy as np

In [91]:
m = GurobiModel()

In [92]:
x = m.add_variables(3, name='gatto')

In [ ]:
import operator


In [94]:
c = m.add_linear_constraints(x, operator.ge, np.zeros(3), name='topo')

In [99]:
aaa.Sense

'>'

In [102]:
vi = 3

In [133]:
d = {'a':0, 'b':1, 'c':2}
for k in d.keys():
    if k == 'b':
        del d[k]
        
print d

{'a': 0, 'c': 2}


In [106]:
c = m.getConstrs

In [108]:
c

<bound method GurobiModel.getConstrs of <gurobi.Model Continuous instance Unnamed: 3 constrs, 3 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>>